In [4]:
import numpy as np
from PIL import Image

# Pytorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# Torchvision
import torchvision
import torchvision.transforms as transforms
from torch.utils.tensorboard.summary import image

In [5]:
transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [6]:
train_data = torchvision.datasets.CIFAR10(
    root='./data',
    train=True,
    download=True,
    transform=transforms)

#test data

test_data = torchvision.datasets.CIFAR10(
    './data',
    train=False,
    download=True,
    transform=transforms
)

#train loader
train_loader = torch.utils.data.DataLoader(
    train_data,
    batch_size=32,
    shuffle=True,
    num_workers=2
)

# test loader
test_loader = torch.utils.data.DataLoader(
    test_data,
    batch_size=32,
    shuffle=False,
    num_workers=2
)

In [7]:
image, label = train_data[0]

In [8]:
image.size()

torch.Size([3, 32, 32])

In [9]:
class_name = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [10]:
class NeuralNet(nn.Module):
    def __init__(self):
        super().__init__()

        self.conv1 = nn.Conv2d(3, 12, 5) # (12, 28, 28)
        self.pool = nn.MaxPool2d(2, 2) # (12, 14, 14)
        self.conv2 = nn.Conv2d(12, 24, 5) # (24, 10, 10) -> (24, 5, 5) -> Flattened to (24 * 5 * 5)
        self.fc1 = nn.Linear(24 * 5 * 5, 120) # (120)
        self.fc2 = nn.Linear(120, 84) # (84)
        self.fc3 = nn.Linear(84, 10) # (10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))  # Apply conv1 and relu, then max pooling
        x = self.pool(F.relu(self.conv2(x)))  # Apply conv2 and relu, then max pooling
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))  # Apply fc1 and relu
        x = F.relu(self.fc2(x))  # Apply fc2 and relu
        x = self.fc3(x)  # Apply fc3 (output layer)
        return x




In [11]:
net = NeuralNet()
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
for epoch in range(30):
    print(f'Training epoch {epoch }')

    running_loss = 0.0

    for i, data in enumerate(train_loader):
        inputs, labels = data

        optimizer.zero_grad()

        outputs = net(inputs)

        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f'Loss: {running_loss / len(train_loader):.4f}')

Training epoch 0
Loss: 1.9187886913236105:.4f
Training epoch 1
Loss: 1.5971928137811575:.4f
Training epoch 2
Loss: 1.445537642721785:.4f
Training epoch 3
Loss: 1.3428730378529237:.4f
Training epoch 4
Loss: 1.2431364970487886:.4f
Training epoch 5
Loss: 1.1691586966706786:.4f
Training epoch 6
Loss: 1.103461619111413:.4f
Training epoch 7
Loss: 1.0572592444284101:.4f
Training epoch 8
Loss: 1.010170953649782:.4f
Training epoch 9
Loss: 0.9701783348754363:.4f
Training epoch 10
Loss: 0.931677785750314:.4f
Training epoch 11
Loss: 0.8955791199428488:.4f
Training epoch 12
Loss: 0.86237318721324:.4f
Training epoch 13
Loss: 0.8328312940309235:.4f
Training epoch 14


In [ ]:
torch.save(net.state_dict(), 'trained_net.pth')

In [ ]:
net = NeuralNet()
net.load_state_dict(torch.load('trained_net.pth'))